# Análisis de resultados

In [1]:
# Importaciones
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from scipy import stats
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal
import re

## 1. Utils

In [2]:
SEED = 9603

bagging_model = [
  {
    'path_name' : 'rf', 
    'model_name' : 'RandomForestClassifier',
  },
  {
    'path_name' : 'dt', 
    'model_name' : 'DecisionTreeClassifier',
  },
  {
    'path_name' : 'mlp',
    'model_name' : 'MLPClassifier',
  },
  {
    'path_name' : 'knn',
    'model_name' : 'KNeighborsClassifier'
  },
]

# Número de modelos 
prng = np.random.RandomState(seed=SEED)
max_int32 = np.iinfo(np.int32).max
SEEDS_POR_MODELO = prng.randint(0, max_int32, size=len(bagging_model))
print(SEEDS_POR_MODELO)

iter_list   = ['Iteración 1', 'Iteración 2', 'Iteración 3', 'Iteración 4', 'Iteración 5', 'Iteración 6', 'Iteración 7', 'Iteración 8', 'Iteración 9', 'Iteración 10']
metric_list = ['Exac', 'Prec', 'Sens', 'F1', 'ms', 'AUC_PS']
label_list  = ['GENERAL', 'BENIGN', 'BOT', 'DDOS', 'DOS_GOLDENEYE', 'DOS_HULK', 'DOS_SLOWHTTPTEST', 'DOS_SLOWLORIS', 'FTP_PATATOR', 'PORTSCAN', 'SSH_PATATOR']
model_list  = ['RandomForestClassifier', 'DecisionTreeClassifier', 'MLPClassifier', 'KNeighborsClassifier', 'IDSBaggingClassifier']
list_models = ['RandomForestClassifier', 'DecisionTreeClassifier', 'MLPClassifier', 'KNeighborsClassifier']
bagging_model_name = 'IDSBaggingClassifier'

[793494059 498241738 377997800 912782427]


In [3]:
# Resultados ordenados por métrica
df_results = pd.read_excel('DB/testing_results.xlsx')
df_results['iter'] = pd.Categorical(df_results['iter'], categories=iter_list, ordered=True)
df_results

,iter,model,label,TP,TN,FP,FN,Exac,Prec,Sens,F1,ms,AUC_PS
0,Iteración 1,RandomForestClassifier,BENIGN,11567,5416,6,4,0.999412,0.999482,0.999654,0.999568,72238.799999,0.999999
1,Iteración 1,RandomForestClassifier,GENERAL,5416,11567,4,6,0.999412,0.999262,0.998893,0.999078,72238.799999,0.999999
2,Iteración 1,RandomForestClassifier,BOT,8,16984,0,1,0.999941,1.000000,0.888889,0.941176,72238.799999,1.000000
3,Iteración 1,RandomForestClassifier,DDOS,1189,15804,0,0,1.000000,1.000000,1.000000,1.000000,72238.799999,1.000000
4,Iteración 1,RandomForestClassifier,DOS_GOLDENEYE,92,16894,5,2,0.999588,0.948454,0.978723,0.963351,72238.799999,0.995983
...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,Iteración 10,IDSBaggingClassifier,DOS_SLOWHTTPTEST,20,16970,0,2,0.999882,1.000000,0.909091,0.952381,466579.700005,0.984790
546,Iteración 10,IDSBaggingClassifier,DOS_SLOWLORIS,50,16941,1,0,0.999941,0.980392,1.000000,0.990099,466579.700005,1.000000
547,Iteración 10,IDSBaggingClassifier,FTP_PATATOR,50,16942,0,0,1.000000,1.000000,1.000000,1.000000,466579.700005,1.000000
548,Iteración 10,IDSBaggingClassifier,PORTSCAN,1989,15002,0,1,0.999941,1.000000,0.999497,0.999749,466579.700005,0.999568


In [4]:
# Resultados ordenados por iteración
df_results_ordered = df_results.copy()
df_results_ordered = df_results_ordered.drop(
  ['TP', 'TN', 'FP', 'FN'], axis=1
).melt(
  id_vars=['iter', 'model', 'label'], var_name='metric', value_name='value'
).pivot(
  index=['metric', 'label', 'model'], columns='iter', values='value'
).reset_index()

# df_results_ordered.columns = ['{}_{}'.format(*col) if isinstance(col, tuple) else col for col in df_results_ordered.columns]
df_results_ordered['metric'] = pd.Categorical(df_results_ordered['metric'], categories=metric_list, ordered=True)
df_results_ordered['label'] = pd.Categorical(df_results_ordered['label'], categories=label_list, ordered=True)
df_results_ordered['model'] = pd.Categorical(df_results_ordered['model'], categories=model_list, ordered=True)

df_results_ordered = df_results_ordered.sort_values(by=['metric', 'label', 'model'])
df_results_ordered

iter,metric,label,model,Iteración 1,Iteración 2,Iteración 3,Iteración 4,Iteración 5,Iteración 6,Iteración 7,Iteración 8,Iteración 9,Iteración 10
99,Exac,GENERAL,RandomForestClassifier,0.999412,0.999529,0.999470,0.999765,0.998411,0.999823,0.999294,0.999647,0.999353,0.998176
95,Exac,GENERAL,DecisionTreeClassifier,0.999058,0.998941,0.998823,0.997940,0.998588,0.999058,0.998588,0.998705,0.997234,0.998823
98,Exac,GENERAL,MLPClassifier,0.989290,0.988995,0.989113,0.988642,0.988584,0.987407,0.988642,0.990467,0.987641,0.989524
97,Exac,GENERAL,KNeighborsClassifier,0.997999,0.998411,0.997999,0.998058,0.998352,0.998293,0.998588,0.998764,0.998529,0.998117
96,Exac,GENERAL,IDSBaggingClassifier,0.999353,0.999353,0.999529,0.999588,0.999588,0.999529,0.999588,0.999470,0.999529,0.999470
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,AUC_PS,SSH_PATATOR,RandomForestClassifier,1.000000,0.999644,1.000000,1.000000,1.000000,1.000000,1.000000,0.995876,1.000000,1.000000
50,AUC_PS,SSH_PATATOR,DecisionTreeClassifier,0.986516,0.973032,0.986842,1.000000,0.948054,1.000000,1.000000,0.960230,0.986516,1.000000
53,AUC_PS,SSH_PATATOR,MLPClassifier,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
52,AUC_PS,SSH_PATATOR,KNeighborsClassifier,0.986842,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.972656,1.000000


In [5]:
with pd.ExcelWriter('output/df_results_ordered_by_iter.xlsx', engine='openpyxl') as writer:
  for metric in metric_list : 
    for label in label_list : 
      df_results_ordered[
        (df_results_ordered['metric'] == metric) &
        (df_results_ordered['label'] == label)
      ].to_excel(writer, sheet_name=f'{metric}_{label}', index=False)

In [6]:
df_results_ordered_by_model = df_results.copy()
df_results_ordered_by_model['model'] = pd.Categorical(df_results_ordered_by_model['model'], categories=model_list, ordered=True)
df_results_ordered_by_model = df_results_ordered_by_model.drop(
  ['TP', 'TN', 'FP', 'FN'], axis=1
).melt(
  id_vars=['iter', 'model', 'label'], var_name='metric', value_name='value'
).pivot(
  index=['metric', 'label', 'iter'], columns='model', values='value'
).reset_index()

# df_results_ordered.columns = ['{}_{}'.format(*col) if isinstance(col, tuple) else col for col in df_results_ordered.columns]
df_results_ordered_by_model['iter'] = pd.Categorical(df_results_ordered_by_model['iter'], categories=iter_list, ordered=True)
df_results_ordered_by_model['metric'] = pd.Categorical(df_results_ordered_by_model['metric'], categories=metric_list, ordered=True)
df_results_ordered_by_model['label'] = pd.Categorical(df_results_ordered_by_model['label'], categories=label_list, ordered=True)

df_results_ordered_by_model = df_results_ordered_by_model.sort_values(by=['metric', 'label', 'iter'])
df_results_ordered_by_model

model,metric,label,iter,RandomForestClassifier,DecisionTreeClassifier,MLPClassifier,KNeighborsClassifier,IDSBaggingClassifier
190,Exac,GENERAL,Iteración 1,0.999412,0.999058,0.989290,0.997999,0.999353
191,Exac,GENERAL,Iteración 2,0.999529,0.998941,0.988995,0.998411,0.999353
192,Exac,GENERAL,Iteración 3,0.999470,0.998823,0.989113,0.997999,0.999529
193,Exac,GENERAL,Iteración 4,0.999765,0.997940,0.988642,0.998058,0.999588
194,Exac,GENERAL,Iteración 5,0.998411,0.998588,0.988584,0.998352,0.999588
...,...,...,...,...,...,...,...,...
105,AUC_PS,SSH_PATATOR,Iteración 6,1.000000,1.000000,1.000000,1.000000,1.000000
106,AUC_PS,SSH_PATATOR,Iteración 7,1.000000,1.000000,1.000000,1.000000,1.000000
107,AUC_PS,SSH_PATATOR,Iteración 8,0.995876,0.960230,1.000000,1.000000,1.000000
108,AUC_PS,SSH_PATATOR,Iteración 9,1.000000,0.986516,1.000000,0.972656,0.999279


In [7]:
with pd.ExcelWriter('output/df_results_ordered_by_model.xlsx', engine='openpyxl') as writer: 
  for metric in metric_list : 
    for label in label_list : 
      df_results_ordered_by_model[
        (df_results_ordered_by_model['metric'] == metric) &
        (df_results_ordered_by_model['label'] == label)
      ].to_excel(writer, sheet_name=f'{metric}_{label}', index=False)

In [8]:
with pd.ExcelWriter('output/df_results_ordered_by_model_describes.xlsx', engine='openpyxl') as writer: 
  for metric in metric_list : 
    for label in label_list : 
      df_results_ordered_by_model[
        (df_results_ordered_by_model['metric'] == metric) &
        (df_results_ordered_by_model['label'] == label)
      ][model_list].describe().to_excel(writer, sheet_name=f'{metric}_{label}', index=False)

In [9]:
def formatear_decimal(numero):
  d = Decimal(str(numero)).quantize(Decimal('1.00000000')).normalize()
  return format(d, 'f')

In [10]:
def formatear_nombre_modelo(nombre):
  texto = nombre[:-10]
  texto = re.sub(r'(?<!^)([A-Z])', r' \1', texto)

  if texto == 'M L P' : 
    return 'Multi Layer Perceptron'
  elif texto == 'K Neighbors' : 
    return 'K-Nearest Neighbor'

  return texto

## Test de Friedmann

In [11]:
""" Test de Friedman """
# H0 (hipótesis nula): No hay diferencias significativas entre los grupos comparados (los modelos rinden igual).
# H1 (hipótesis alternativa): Al menos un grupo difiere significativamente de los demás.
# Nivel de significancia = 0.05
# Si p <= nivel de significancia  ->  Se rechaza H0 (al menos una muestra difiere de las demás)

df_test_friedmann = pd.DataFrame(columns=["metric", "label", "gl", "stadistic", "p_value", "las muestras difieren?"])
nivel_significancia = 0.05

for metric in metric_list : 
  for label in label_list :
    lista_friedman = []
    for model in model_list : 
      mask = (
        (df_results_ordered['metric'] == metric) & 
        (df_results_ordered['label'] == label) & 
        (df_results_ordered['model'] == model)
      )
      muestra = df_results_ordered[mask][iter_list].iloc[0].to_list()
      lista_friedman.append(muestra)
    stat, p = stats.friedmanchisquare(*lista_friedman)
    df_test_friedmann.loc[len(df_test_friedmann)] = [
      metric,
      label,
      len(lista_friedman[0])-1,
      stat,
      p,
      p <= nivel_significancia 
    ]

df_test_friedmann

,metric,label,gl,stadistic,p_value,las muestras difieren?
0,Exac,GENERAL,9,33.708543,8.551161e-07,True
1,Exac,BENIGN,9,33.708543,8.551161e-07,True
2,Exac,BOT,9,35.068783,4.496648e-07,True
3,Exac,DDOS,9,16.671329,2.238784e-03,True
4,Exac,DOS_GOLDENEYE,9,33.676768,8.680372e-07,True
...,...,...,...,...,...,...
61,AUC_PS,DOS_SLOWHTTPTEST,9,27.051546,1.940754e-05,True
62,AUC_PS,DOS_SLOWLORIS,9,33.409326,9.847922e-07,True
63,AUC_PS,FTP_PATATOR,9,12.037736,1.707285e-02,True
64,AUC_PS,PORTSCAN,9,26.111111,3.005331e-05,True


## Test de Shapiro-Wilk

In [12]:
# Prueba de Shapiro Wilk sobre cada muestra
""" Test de Shapiro Wilk """
# H0 (hipótesis nula): Los datos provienen de una distribución normal
# H1 (hipótesis alternativa): Los datos no provienen de una distribución normal
# Nivel de significancia = 0.05
# Si p < nivel de significancia  ->  Los datos no provienen de una distribución normal
df_shapiro_wilk = pd.DataFrame(columns=["metric", "label", "model", "stadistic", "p_value", "es fiable?"])
nivel_significancia_normalidad = 0.05

for metric in metric_list : 
  for label in label_list :
    for model in [bagging_model_name]+list_models :
      muestra = df_results_ordered[
        (df_results_ordered['metric'] == metric) & 
        (df_results_ordered['label'] == label) & 
        (df_results_ordered['model'] == model)
      ][iter_list].iloc[0].to_list()

      fiable = True
      if np.ptp(muestra) > 0 : 
        fiable = True
      else : 
        print(f'Los resultados de la prueba en {metric}, {label} y {model} no son fiables porque tienen varianza 0')
        fiable = False

      stat, p = stats.shapiro(muestra)

      df_shapiro_wilk.loc[len(df_shapiro_wilk)] = [
        metric,
        label,
        model,
        stat,
        p,
        fiable
      ]
reject, p_corrected, _, _ = multipletests(
  df_shapiro_wilk['p_value'].to_list(),
  alpha=nivel_significancia_normalidad,
  method='holm'
) 

df_shapiro_wilk['p_corrected'] = p_corrected
df_shapiro_wilk['reject'] = reject
df_shapiro_wilk[df_shapiro_wilk['reject'] & df_shapiro_wilk['es fiable?']]

Los resultados de la prueba en Prec, SSH_PATATOR y IDSBaggingClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en Prec, SSH_PATATOR y RandomForestClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en Sens, DDOS y DecisionTreeClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en Sens, FTP_PATATOR y IDSBaggingClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en AUC_PS, DDOS y IDSBaggingClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en AUC_PS, DDOS y RandomForestClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en AUC_PS, FTP_PATATOR y IDSBaggingClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en AUC_PS, FTP_PATATOR y RandomForestClassifier no son fiables porque tienen varianza 0


c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_a

,metric,label,model,stadistic,p_value,es fiable?,p_corrected,reject
11,Exac,BOT,RandomForestClassifier,0.594183,4.714743e-05,True,0.013386,True
15,Exac,DDOS,IDSBaggingClassifier,0.594183,4.714743e-05,True,0.013386,True
16,Exac,DDOS,RandomForestClassifier,0.531644,8.563494e-06,True,0.002492,True
17,Exac,DDOS,DecisionTreeClassifier,0.365721,1.003693e-07,True,0.000033,True
40,Exac,FTP_PATATOR,IDSBaggingClassifier,0.365721,1.003693e-07,True,0.000033,True
...,...,...,...,...,...,...,...,...
320,AUC_PS,PORTSCAN,IDSBaggingClassifier,0.373213,1.224219e-07,True,0.000038,True
321,AUC_PS,PORTSCAN,RandomForestClassifier,0.572053,2.572848e-05,True,0.007384,True
325,AUC_PS,SSH_PATATOR,IDSBaggingClassifier,0.365721,1.003693e-07,True,0.000033,True
326,AUC_PS,SSH_PATATOR,RandomForestClassifier,0.407557,3.050081e-07,True,0.000095,True


In [13]:
# Prueba de Shapiro Wilk sobre diferencia de muestras pareadas
""" Test de Shapiro Wilk """
# H0 (hipótesis nula): Los datos provienen de una distribución normal
# H1 (hipótesis alternativa): Los datos no provienen de una distribución normal
# Nivel de significancia = 0.05
# Si p < nivel de significancia  ->  Los datos no provienen de una distribución normal
df_shapiro_wilk_diferencias = pd.DataFrame(columns=["metric", "label", "model", "difference", "stadistic", "p_value", "es fiable?"])
nivel_significancia_normalidad = 0.05

for metric in metric_list : 
  for label in label_list :
    mask = (df_results_ordered['metric'] == metric) & (df_results_ordered['label'] == label)
    bagging_mask = mask & (df_results_ordered['model'] == bagging_model_name)
    muestra_bagging = df_results_ordered[bagging_mask][iter_list].iloc[0]
    for model in list_models :
      model_mask = mask & (df_results_ordered['model'] == model)
      muestra = df_results_ordered[model_mask][iter_list].iloc[0]
      muestra_diferencia = muestra_bagging-muestra
      fiable = True
      if np.ptp(muestra_diferencia) > 0 : 
        fiable = True
      else : 
        print(f'Los resultados de la prueba en {metric}, {label} y {model} no son fiables porque tienen varianza 0')
        fiable = False

      stat, p = stats.shapiro(muestra_diferencia.to_list())

      df_shapiro_wilk_diferencias.loc[len(df_shapiro_wilk_diferencias)] = [
        metric,
        label,
        model,
        f'{bagging_model_name} - {model}',
        stat,
        p,
        fiable,
      ]
reject, p_corrected, _, _ = multipletests(
  df_shapiro_wilk_diferencias['p_value'].to_list(),
  alpha=nivel_significancia_normalidad,
  method='holm'
) 

df_shapiro_wilk_diferencias['p_corrected'] = p_corrected
df_shapiro_wilk_diferencias['reject'] = reject
df_shapiro_wilk_diferencias['test'] = np.where(
    (~df_shapiro_wilk_diferencias['reject']) & (df_shapiro_wilk_diferencias['es fiable?']),
    'T de Student',
    'Wilcoxon'
)
def interpretar_fila(row):
    if not row['es fiable?']:
        return f"No se puede realizar la prueba de normalidad de Shapiro-Wilk debido a que las diferencias entre los pares tienen varianza 0, asimismo, se interpreta que los resultados son iguales en cada par"
    elif not row['reject']:
        return f"Se realizó la prueba de normalidad de Shapiro-Wilk sobre las diferencias entre los pares del metamodelo de ensamble {'"Bagging"'} y el algoritmo {formatear_nombre_modelo(row['model'])}, obteniéndose un estadístico W {formatear_decimal(row['stadistic'])} y un valor p corregido {formatear_decimal(row['p_corrected'])} mayor al nivel de significancia ({nivel_significancia_normalidad}), por ello, no se rechaza la hipótesis nula y se asume que las diferencias entre los pares siguen una distribución normal, asimismo, se optará por el uso de la prueba T de Student."
    else:
        return f"Se realizó la prueba de normalidad de Shapiro-Wilk sobre las diferencias entre los pares del metamodelo de ensamble {'"Bagging"'} y el algoritmo {formatear_nombre_modelo(row['model'])}, obteniéndose un estadístico W {formatear_decimal(row['stadistic'])} y un valor p corregido {formatear_decimal(row['p_corrected'])} menor al nivel de significancia ({nivel_significancia_normalidad}), por ello, se rechaza la hipótesis nula y se asume que las diferencias entre los pares no siguen una distribución normal, asimismo, se optará por el uso de la prueba Wilcoxon."

df_shapiro_wilk_diferencias['interpretacion'] = df_shapiro_wilk_diferencias.apply(interpretar_fila, axis=1)
df_shapiro_wilk_diferencias = df_shapiro_wilk_diferencias[['metric', 'label', 'model', 'es fiable?', 'reject', 'difference', 'stadistic', 'p_value', 'p_corrected', 'interpretacion', 'test']]
df_shapiro_wilk_diferencias.to_excel('output/df_shapiro_wilk.xlsx', index=False)
df_shapiro_wilk_diferencias

Los resultados de la prueba en Prec, FTP_PATATOR y RandomForestClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en Prec, SSH_PATATOR y RandomForestClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en Sens, BOT y DecisionTreeClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en AUC_PS, DDOS y RandomForestClassifier no son fiables porque tienen varianza 0
Los resultados de la prueba en AUC_PS, FTP_PATATOR y RandomForestClassifier no son fiables porque tienen varianza 0


c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_a

,metric,label,model,es fiable?,reject,difference,stadistic,p_value,p_corrected,interpretacion,test
0,Exac,GENERAL,RandomForestClassifier,True,False,IDSBaggingClassifier - RandomForestClassifier,0.777261,7.666488e-03,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
1,Exac,GENERAL,DecisionTreeClassifier,True,False,IDSBaggingClassifier - DecisionTreeClassifier,0.856477,6.933510e-02,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
2,Exac,GENERAL,MLPClassifier,True,False,IDSBaggingClassifier - MLPClassifier,0.964717,8.380139e-01,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
3,Exac,GENERAL,KNeighborsClassifier,True,False,IDSBaggingClassifier - KNeighborsClassifier,0.939242,5.446063e-01,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
4,Exac,BENIGN,RandomForestClassifier,True,False,IDSBaggingClassifier - RandomForestClassifier,0.777261,7.666488e-03,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
...,...,...,...,...,...,...,...,...,...,...,...
259,AUC_PS,PORTSCAN,KNeighborsClassifier,True,False,IDSBaggingClassifier - KNeighborsClassifier,0.919986,3.568414e-01,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
260,AUC_PS,SSH_PATATOR,RandomForestClassifier,True,True,IDSBaggingClassifier - RandomForestClassifier,0.516595,5.695451e-06,0.001412,Se realizó la prueba de normalidad de Shapiro-...,Wilcoxon
261,AUC_PS,SSH_PATATOR,DecisionTreeClassifier,True,False,IDSBaggingClassifier - DecisionTreeClassifier,0.837377,4.104164e-02,1.000000,Se realizó la prueba de normalidad de Shapiro-...,T de Student
262,AUC_PS,SSH_PATATOR,MLPClassifier,True,True,IDSBaggingClassifier - MLPClassifier,0.365721,1.003693e-07,0.000026,Se realizó la prueba de normalidad de Shapiro-...,Wilcoxon


## Test T de Student y Wilcoxon

In [14]:
# Prueba T de Student y Wilcoxon
""" Test T de Student unilateral """
# H0 (hipótesis nula): El metamodelo de ensamble Bagging tiene resultados iguales o menores a la instancia evaluada
# H1 (hipótesis alternativa): El metamodelo de ensamble Bagging tiene mejores resultados que la instancia evaluada
# Nivel de significancia = 0.05
# Si p < nivel de significancia  ->  Se acepta la hipótesis nula
""" Test Wilcoxon """
# H0 (hipótesis nula): El metamodelo de ensamble Bagging tiene resultados iguales o menores a la instancia evaluada
# H1 (hipótesis alternativa): El metamodelo de ensamble Bagging tiene mejores resultados que la instancia evaluada
# Nivel de significancia = 0.05
# Si p < nivel de significancia  ->  Se acepta la hipótesis nula
df_prueba_estadistica_diferencias_significativas = pd.DataFrame(columns=['metric', 'label', 'model', 'comparison', 'test', 'stat', 'p_value'])
nivel_significancia_diferencias_significativas = 0.05
for metric in metric_list : 
  for label in label_list :
    mask = (df_results_ordered['metric'] == metric) & (df_results_ordered['label'] == label)
    bagging_mask = mask & (df_results_ordered['model'] == bagging_model_name)
    muestra_bagging = df_results_ordered[bagging_mask][iter_list].iloc[0]
    for model in list_models :
      model_mask = mask & (df_results_ordered['model'] == model)
      muestra = df_results_ordered[model_mask][iter_list].iloc[0]
      test = df_shapiro_wilk_diferencias[
        (df_shapiro_wilk_diferencias['metric'] == metric) & 
        (df_shapiro_wilk_diferencias['label'] == label) & 
        (df_shapiro_wilk_diferencias['model'] == model)
      ]['test'].iloc[0]
      if test == 'T de Student' : 
        stat, p_value = stats.ttest_rel(muestra_bagging, muestra, alternative='greater')
        df_prueba_estadistica_diferencias_significativas.loc[len(df_prueba_estadistica_diferencias_significativas)] = [
          metric,
          label,
          model,
          f'{bagging_model_name} > {model}',
          test,
          stat,
          p_value
        ]
      elif test == 'Wilcoxon' : 
        stat, p_value = stats.wilcoxon(muestra_bagging, muestra, alternative='greater')
        df_prueba_estadistica_diferencias_significativas.loc[len(df_prueba_estadistica_diferencias_significativas)] = [
          metric,
          label,
          model,
          f'{bagging_model_name} > {model}',
          test,
          stat,
          p_value
        ]
reject, p_corrected, _, _ = multipletests(
  df_prueba_estadistica_diferencias_significativas['p_value'].to_list(),
  alpha=nivel_significancia_diferencias_significativas,
  method='holm'
) 
df_prueba_estadistica_diferencias_significativas['es fiable?'] = df_shapiro_wilk_diferencias['es fiable?']
df_prueba_estadistica_diferencias_significativas['p_corrected'] = p_corrected
df_prueba_estadistica_diferencias_significativas['reject'] = reject

def interpretar_fila(row):
    if not row['es fiable?']:
        return f"No se realizó la prueba para diferencia de medias debido a que las diferencias entre los pares tienen varianza 0."
    elif not row['reject']:
        return f"Se realizó la prueba de {row['test']} sobre las diferencias entre los pares del metamodelo de ensamble {'"Bagging"'} y el algoritmo {formatear_nombre_modelo(row['model'])}, obteniéndose un estadístico {formatear_decimal(row['stat'])} y un valor p {formatear_decimal(row['p_corrected'])} mayor al nivel de significancia ({nivel_significancia_diferencias_significativas}), por ello, se rechaza la hipótesis nula y se asume que el metamodelo de ensamble Bagging tiene resultados iguales o peores que el algoritmo {formatear_nombre_modelo(row['model'])}."
    else:
        return f"Se realizó la prueba de {row['test']} sobre las diferencias entre los pares del metamodelo de ensamble {'"Bagging"'} y el algoritmo {formatear_nombre_modelo(row['model'])}, obteniéndose un estadístico {formatear_decimal(row['stat'])} y un valor p {formatear_decimal(row['p_corrected'])} menor al nivel de significancia ({nivel_significancia_diferencias_significativas}), por ello, no se rechaza la hipótesis nula y se asume que el metamodelo de ensamble Bagging tiene mejores resultados que el algoritmo {formatear_nombre_modelo(row['model'])}."

df_prueba_estadistica_diferencias_significativas['interpretacion'] = df_prueba_estadistica_diferencias_significativas.apply(interpretar_fila, axis=1)

df_prueba_estadistica_diferencias_significativas = df_prueba_estadistica_diferencias_significativas[['metric', 'label', 'model', 'es fiable?', 'reject', 'test', 'comparison', 'stat', 'p_value', 'p_corrected', 'interpretacion']]
df_prueba_estadistica_diferencias_significativas.to_excel('output/df_diferencias_significativas.xlsx')
df_prueba_estadistica_diferencias_significativas

c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se


,metric,label,model,es fiable?,reject,test,comparison,stat,p_value,p_corrected,interpretacion
0,Exac,GENERAL,RandomForestClassifier,True,False,T de Student,IDSBaggingClassifier > RandomForestClassifier,1.176712,1.347469e-01,1.000000e+00,Se realizó la prueba de T de Student sobre las...
1,Exac,GENERAL,DecisionTreeClassifier,True,False,T de Student,IDSBaggingClassifier > DecisionTreeClassifier,4.733337,5.344395e-04,8.016593e-02,Se realizó la prueba de T de Student sobre las...
2,Exac,GENERAL,MLPClassifier,True,True,T de Student,IDSBaggingClassifier > MLPClassifier,36.501759,2.153056e-11,4.693663e-09,Se realizó la prueba de T de Student sobre las...
3,Exac,GENERAL,KNeighborsClassifier,True,True,T de Student,IDSBaggingClassifier > KNeighborsClassifier,13.917370,1.079215e-07,2.212392e-05,Se realizó la prueba de T de Student sobre las...
4,Exac,BENIGN,RandomForestClassifier,True,False,T de Student,IDSBaggingClassifier > RandomForestClassifier,1.176712,1.347469e-01,1.000000e+00,Se realizó la prueba de T de Student sobre las...
...,...,...,...,...,...,...,...,...,...,...,...
259,AUC_PS,PORTSCAN,KNeighborsClassifier,True,False,T de Student,IDSBaggingClassifier > KNeighborsClassifier,2.075635,3.387134e-02,1.000000e+00,Se realizó la prueba de T de Student sobre las...
260,AUC_PS,SSH_PATATOR,RandomForestClassifier,True,False,Wilcoxon,IDSBaggingClassifier > RandomForestClassifier,14.000000,7.109375e-01,1.000000e+00,Se realizó la prueba de Wilcoxon sobre las dif...
261,AUC_PS,SSH_PATATOR,DecisionTreeClassifier,True,False,T de Student,IDSBaggingClassifier > DecisionTreeClassifier,2.726427,1.168037e-02,1.000000e+00,Se realizó la prueba de T de Student sobre las...
262,AUC_PS,SSH_PATATOR,MLPClassifier,True,False,Wilcoxon,IDSBaggingClassifier > MLPClassifier,5.000000,9.843750e-01,1.000000e+00,Se realizó la prueba de Wilcoxon sobre las dif...


In [15]:
with pd.ExcelWriter('output/df_diferencias_significativas_resumen.xlsx', engine='openpyxl') as writer:
  for metric in metric_list : 
    df_pruebita = df_prueba_estadistica_diferencias_significativas[
        df_prueba_estadistica_diferencias_significativas['metric'] == metric
    ][['label', 'comparison', 'es fiable?', 'reject']].copy()
    
    def interpretar_fila(row):
        if not row['es fiable?']:
            return "No"
        elif not row['reject']:
            return "No"
        else:
            return "Sí"
    df_pruebita['value'] = df_pruebita.apply(interpretar_fila, axis=1)
    df_pruebita['label'] = pd.Categorical(
        df_pruebita['label'], 
        categories=label_list,
        ordered=True
    )
    df_pruebita['comparison'] = pd.Categorical(
        df_pruebita['comparison'], 
        categories=['IDSBaggingClassifier > RandomForestClassifier', 'IDSBaggingClassifier > DecisionTreeClassifier', 'IDSBaggingClassifier > MLPClassifier', 'IDSBaggingClassifier > KNeighborsClassifier'],
        ordered=True
    )
    df_pruebita = df_pruebita.drop(['es fiable?', 'reject'], axis=1)
    
    df_pruebita = df_pruebita.pivot(index='label', columns='comparison', values='value').reset_index()
    df_pruebita.to_excel(writer, sheet_name=f'{metric}', index=False)